In [1]:
#!pip install iterative-stratification
#!pip install ensemble-boxes
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from ensemble_boxes import nms, weighted_boxes_fusion

In [2]:
import yaml

import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
import shutil
from tqdm.notebook import tqdm
import seaborn as sns

In [17]:
# 設定

CFG = {
    "dim" : 512,  #512, 1024
    "way" : "_hist", # "", "_hist", "_clahe"
    "fold_num" : 5,
    "seed": 11,
    "bach_size": 32,
    "weights": "yolov5m.pt",
    "hyp_conf": "hyp00",
    "epochs": 60
}
list_remove = []
image_remove = []

MAIN_PATH = '../input/vinbigdata-chest-xray-abnormalities-detection/'
YOLO_DATA_DIR = '/home/kaggle-vinbigdata-xray/working/chest_yolo' # yolov5 ディレクトリから見た位置
SUB_PATH = os.path.join(MAIN_PATH, 'sample_submission.csv')
TRAIN_PATH = f'../input/vinbigdata-chest-xray-resized-png-{CFG["dim"]}x{CFG["dim"]}{CFG["way"]}/train'
TRAIN_META_PATH = f'../input/vinbigdata-chest-xray-resized-png-{CFG["dim"]}x{CFG["dim"]}/train_meta.csv'
TEST_META_PATH = '../input/vinbigdata-testmeta/test_meta.csv'

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |


In [90]:
train_df = pd.read_csv(os.path.join(MAIN_PATH,'train.csv'))
print("train_img_dir:", TRAIN_PATH)
train_df['image_path'] = f'{TRAIN_PATH}'+train_df.image_id+'.png'
train_meta_df = pd.read_csv(TRAIN_META_PATH)
train_meta_df.columns = ['image_id', 'h', 'w']
train_df = train_df.merge(train_meta_df, on="image_id")
display(train_df.head())
print(train_df.shape)

train_img_dir: ../input/vinbigdata-chest-xray-resized-png-512x512_hist


,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,image_path,h,w
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,../input/vinbigdata-chest-xray-resized-png-512...,2580,2332
1,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R15,NaN,NaN,NaN,NaN,../input/vinbigdata-chest-xray-resized-png-512...,2580,2332
2,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R16,NaN,NaN,NaN,NaN,../input/vinbigdata-chest-xray-resized-png-512...,2580,2332
3,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,../input/vinbigdata-chest-xray-resized-png-512...,3159,2954
4,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R13,NaN,NaN,NaN,NaN,../input/vinbigdata-chest-xray-resized-png-512...,3159,2954


(67914, 11)


## preprocessing
使用する dim でのサイズに box を変換する

In [91]:
def label_resize(org_size, img_size, *bbox):
    x0, y0, x1, y1 = bbox
    x0_new = int(np.round(x0*img_size[1]/org_size[1]))
    y0_new = int(np.round(y0*img_size[0]/org_size[0]))
    x1_new = int(np.round(x1*img_size[1]/org_size[1]))
    y1_new = int(np.round(y1*img_size[0]/org_size[0]))
    return x0_new, y0_new, x1_new, y1_new

train_normal = train_df[train_df['class_name']=='No finding'].reset_index(drop=True)
train_normal['x_min_resize'] = 0
train_normal['y_min_resize'] = 0
train_normal['x_max_resize'] = 1
train_normal['y_max_resize'] = 1

train_abnormal = train_df[train_df['class_name']!='No finding'].reset_index(drop=True)
train_abnormal[['x_min_resize', 'y_min_resize', 'x_max_resize', 'y_max_resize']] = train_abnormal \
.apply(lambda x: label_resize(x[['h', 'w']].values, [CFG["dim"],CFG["dim"]], *x[['x_min', 'y_min', 'x_max', 'y_max']].values),
       axis=1, result_type="expand")
train_abnormal['x_center'] = 0.5*(train_abnormal['x_min_resize'] + train_abnormal['x_max_resize'])
train_abnormal['y_center'] = 0.5*(train_abnormal['y_min_resize'] + train_abnormal['y_max_resize'])
train_abnormal['width'] = train_abnormal['x_max_resize'] - train_abnormal['x_min_resize']
train_abnormal['height'] = train_abnormal['y_max_resize'] - train_abnormal['y_min_resize']
train_abnormal['area'] = train_abnormal.apply(lambda x: (x['x_max_resize']-x['x_min_resize'])*(x['y_max_resize']-x['y_min_resize']), axis=1)
train_abnormal = train_abnormal[~train_abnormal.index.isin(list_remove)].reset_index(drop=True)

train_abnormal.tail()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,image_path,h,w,x_min_resize,y_min_resize,x_max_resize,y_max_resize,x_center,y_center,width,height,area
35894,52951d7de2485aba8ed62629eee4d254,Other lesion,9,R9,303.0,1442.0,383.0,1508.0,../input/vinbigdata-chest-xray-resized-png-512...,2880,2304,67,256,85,268,76.0,262.0,18,12,216
35895,52951d7de2485aba8ed62629eee4d254,Cardiomegaly,3,R8,734.0,1571.0,1614.0,1892.0,../input/vinbigdata-chest-xray-resized-png-512...,2880,2304,163,279,359,336,261.0,307.5,196,57,11172
35896,1224f07d895107573588225f692e94f9,Aortic enlargement,0,R10,999.0,716.0,1276.0,988.0,../input/vinbigdata-chest-xray-resized-png-512...,2264,2040,251,162,320,223,285.5,192.5,69,61,4209
35897,1224f07d895107573588225f692e94f9,Aortic enlargement,0,R8,1046.0,688.0,1272.0,979.0,../input/vinbigdata-chest-xray-resized-png-512...,2264,2040,263,156,319,221,291.0,188.5,56,65,3640
35898,1224f07d895107573588225f692e94f9,Aortic enlargement,0,R9,1036.0,681.0,1271.0,966.0,../input/vinbigdata-chest-xray-resized-png-512...,2264,2040,260,154,319,218,289.5,186.0,59,64,3776


## change by wbf

In [92]:
def Preprocess_wbf(df, size=CFG["dim"], iou_thr=0.5, skip_box_thr=0.0001):
    list_image = []
    list_boxes = []
    list_cls = []
    list_h, list_w = [], []
    new_df = pd.DataFrame()
    for image_id in tqdm(df['image_id'].unique(), leave=False):
        image_df = df[df['image_id']==image_id].reset_index(drop=True)
        h, w = image_df.loc[0, ['h', 'w']].values
        boxes = image_df[['x_min_resize', 'y_min_resize',
                          'x_max_resize', 'y_max_resize']].values.tolist()
        boxes = [[j/(size-1) for j in i] for i in boxes]
        scores = [1.0]*len(boxes)
        labels = [float(i) for i in image_df['class_id'].values]
        boxes, scores, labels = weighted_boxes_fusion([boxes], [scores], [labels],
                                                      weights=None,
                                                      iou_thr=iou_thr,
                                                      skip_box_thr=skip_box_thr)
        list_image.extend([image_id]*len(boxes))
        list_h.extend([h]*len(boxes))
        list_w.extend([w]*len(boxes))
        list_boxes.extend(boxes)
        list_cls.extend(labels.tolist())
    list_boxes = [[int(j*(size-1)) for j in i] for i in list_boxes]
    new_df['image_id'] = list_image
    new_df['class_id'] = list_cls
    new_df['h'] = list_h
    new_df['w'] = list_w
    new_df['x_min_resize'], new_df['y_min_resize'], \
    new_df['x_max_resize'], new_df['y_max_resize'] = np.transpose(list_boxes)
    new_df['x_center'] = 0.5*(new_df['x_min_resize'] + new_df['x_max_resize'])
    new_df['y_center'] = 0.5*(new_df['y_min_resize'] + new_df['y_max_resize'])
    new_df['width'] = new_df['x_max_resize'] - new_df['x_min_resize']
    new_df['height'] = new_df['y_max_resize'] - new_df['y_min_resize']
    new_df['area'] = new_df.apply(lambda x: (x['x_max_resize']-x['x_min_resize'])\
                                  *(x['y_max_resize']-x['y_min_resize']), axis=1)
    return new_df

train_abnormal = Preprocess_wbf(train_abnormal)
train_abnormal.tail()

/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


,image_id,class_id,h,w,x_min_resize,y_min_resize,x_max_resize,y_max_resize,x_center,y_center,width,height,area
23883,be53fe5a49231f1c1be020b0bdd8561f,6.0,2880,2304,98,196,149,226,123.5,211.0,51,30,1530
23884,380d07a94cc4b012812119370de47192,0.0,2368,1864,288,142,354,199,321.0,170.5,66,57,3762
23885,52951d7de2485aba8ed62629eee4d254,9.0,2880,2304,67,256,85,268,76.0,262.0,18,12,216
23886,52951d7de2485aba8ed62629eee4d254,3.0,2880,2304,160,286,359,340,259.5,313.0,199,54,10746
23887,1224f07d895107573588225f692e94f9,0.0,2264,2040,258,157,319,220,288.5,188.5,61,63,3843


## split for CV

In [112]:
def split_df(df):
    kf = MultilabelStratifiedKFold(n_splits=CFG["fold_num"], shuffle=True, random_state=CFG["seed"])
    df['id'] = df.index
    annot_pivot = pd.pivot_table(df, index=['image_id'], columns=['class_id'],
                                 values='id', fill_value=0, aggfunc='count') \
    .reset_index().rename_axis(None, axis=1)
    for fold, (train_idx, val_idx) in enumerate(kf.split(annot_pivot,
                                                         annot_pivot.iloc[:, 1:(1+df['class_id'].nunique())])):
        annot_pivot[f'fold_{fold}'] = 0
        annot_pivot.loc[val_idx, f'fold_{fold}'] = 1
    return annot_pivot

size_df = pd.read_csv(TRAIN_META_PATH)
size_df.columns = ['image_id', 'h', 'w']

fold_csv = split_df(train_df)
fold_csv = fold_csv.merge(size_df, on='image_id', how='left')
display(fold_csv.head())
print(fold_csv.shape)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=11 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


,image_id,0,1,2,3,4,5,6,7,8,...,12,13,14,fold_0,fold_1,fold_2,fold_3,fold_4,h,w
0,000434271f63a053c4128a0ba6352c7f,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,1,0,0,2836,2336
1,00053190460d56c53cc3e57321387478,0,0,0,0,0,0,0,0,0,...,0,0,3,1,0,0,0,0,2430,1994
2,0005e8e3701dfb1dd93d53e2ff537b6e,0,0,0,0,1,0,1,2,1,...,0,0,0,0,1,0,0,0,3072,3072
3,0006e0a85696f6bb578e84fafa9a5607,0,0,0,0,0,0,0,0,0,...,0,0,3,0,1,0,0,0,3000,3000
4,0007d316f756b3fa0baea2ff514ce945,2,0,0,1,0,2,0,0,0,...,0,3,0,0,1,0,0,0,2880,2304


(15000, 23)


## create dataset (yolo 用の設定ファイルを作る）
- train, val に分けた image (input 内にある) と label (新しくchest_yolo/labels に作る） へのパスを記したファイル
- 上記のファイルを記した yml ファイル

In [150]:

images_dir = f'{YOLO_DATA_DIR}/images' # yolo は images を自動的に labels に置換して label のファイルを認識する
labels_dir = f'{YOLO_DATA_DIR}/labels'

def create_labels(df, split_df, train_folder, size = CFG["dim"]):
    """
    全てのimageとラベルを作成する
    """
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    
    for image_id in tqdm(split_df.image_id.unique()):
        label_path = f'{labels_dir}/{image_id}.txt'
        with open(label_path, 'w+') as f:
            row = df[df['image_id']==image_id][['class_id', 'x_center', 'y_center', 'width', 'height']].values
            row[:, 1:] /= size
            row = row.astype('str')
            for box in range(len(row)):
                text = ' '.join(row[box])
                f.write(text)
                f.write('\n')
        image_path = f'{images_dir}/{image_id}.png'
        shutil.copy(f'{train_folder}/{image_id}.png', image_path)

create_labels(train_abnormal, fold_csv, TRAIN_PATH)
# img へのパスを記す
fold_csv["img_path"] = images_dir + "/" + fold_csv["image_id"] + ".png"
fold_csv.head()

,image_id,0,1,2,3,4,5,6,7,8,...,14,fold_0,fold_1,fold_2,fold_3,fold_4,h,w,img_path,label_path
0,000434271f63a053c4128a0ba6352c7f,0,0,0,0,0,0,0,0,0,...,3,0,0,1,0,0,2836,2336,/home/kaggle-vinbigdata-xray/working/chest_yol...,./chest_yolo/labels/000434271f63a053c4128a0ba6...
1,00053190460d56c53cc3e57321387478,0,0,0,0,0,0,0,0,0,...,3,1,0,0,0,0,2430,1994,/home/kaggle-vinbigdata-xray/working/chest_yol...,./chest_yolo/labels/00053190460d56c53cc3e57321...
2,0005e8e3701dfb1dd93d53e2ff537b6e,0,0,0,0,1,0,1,2,1,...,0,0,1,0,0,0,3072,3072,/home/kaggle-vinbigdata-xray/working/chest_yol...,./chest_yolo/labels/0005e8e3701dfb1dd93d53e2ff...
3,0006e0a85696f6bb578e84fafa9a5607,0,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,3000,3000,/home/kaggle-vinbigdata-xray/working/chest_yol...,./chest_yolo/labels/0006e0a85696f6bb578e84fafa...
4,0007d316f756b3fa0baea2ff514ce945,2,0,0,1,0,2,0,0,0,...,0,0,1,0,0,0,2880,2304,/home/kaggle-vinbigdata-xray/working/chest_yol...,./chest_yolo/labels/0007d316f756b3fa0baea2ff51...


In [151]:

def create_path_file(split_df,  yolo_data_dir, fold):
    """
    train, val へのパスを記したファイルを fold ごとに作り、それらに対応するymlファイルをを作る
    """
    train_df = split_df[split_df[f'fold_{fold}']!=0].reset_index(drop=True)
    val_df = split_df[split_df[f'fold_{fold}']==0].reset_index(drop=True)
    
    train_file_list = f"{yolo_data_dir}/train_list_fold_{fold}.txt"
    val_file_list = f"{yolo_data_dir}/train_list_fold_{fold}.txt"
    train_df["img_path"].to_csv(train_file_list, header=False, index=False)
    val_df["img_path"].to_csv(val_file_list, header=False, index=False)
    
    # train.py にわたす設定ファイル
    data = dict(
        train =  train_file_list ,
        val   =  val_file_list,
        nc    = 14,
        names = [f"{i}" for i in range(14)] 
        )
    yaml_file = os.path.join(yolo_data_dir, f'yolo_{fold}.yaml')

    with open(yaml_file, 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)
    f = open(yaml_file, 'r')
    f.close()
    
for f in range(CFG["fold_num"]):
    create_path_file(fold_csv, YOLO_DATA_DIR, fold=f)


## train

In [103]:
"""
!git clone https://github.com/ultralytics/yolov5
%pip install pycocotools -qr yolov5/requirements.txt  # install dependencies
%pip uninstall -y wandb  # open wandb bugs 
"""

fatal: destination path 'yolov5' already exists and is not an empty directory.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
os.chdir('yolov5')

In [18]:
fold = 0
exp_name = f'{CFG["hyp_conf"]}_{fold}'
hyp_conf_file = f'../chest_yolo/{CFG["hyp_conf"]}.yaml'
data_file = f"../chest_yolo/yolo_{fold}.yaml"

epochs = CFG["epochs"]
bach_size = CFG["bach_size"]
weights  = CFG["weights"]
dim = CFG["dim"]

!python train.py \
 --epochs $epochs \
 --batch-size $bach_size \
 --hyp $hyp_conf_file \
 --data $data_file \
 --weights $weights \
 --img $dim \
 --name $exp_name

github: up to date with https://github.com/ultralytics/yolov5 ✅
2021-03-16 06:31:21.796752: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
train: Scanning '/home/kaggle-vinbigdata-xray/working/chest_yolo/train_list_fold
val: Scanning '/home/kaggle-vinbigdata-xray/working/chest_yolo/train_list_fold_0
Plotting labels... 

autoanchor: Analyzing anchors... anchors/target = 5.06, Best Possible Recall (BPR) = 0.9994
      0/59     4.82G    0.0836    0.0155   0.05047    0.1496        21       512
               Class      Images      Labels           P           R      mAP@.5
                 all       11986       19100       0.881      0.0528      0.0211     0.00431
      1/59     7.67G   0.07219   0.01285   0.03543    0.1205        21       512
               Class      Images      Labels           P           R      mAP@.5
                 all       11986       19100       0.888      0.0974       0.102      0.0351
     

In [148]:
os.chdir('..')
!ls

config	logs  output  preprocessing  utils  yolo_start.ipynb  yolov5
